# Timing Exercises

In the following exercises, we will practice some basics of timing analysis.


In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt

import numpy as np

## 1. Simulating white noise 

As a simple warm-up exercise, let's look at the power spectrum of white noise. White noise is a process without variability: a constant flux with time, modified by some kind of statistical uncertainty due to the details of the detector. 

Let's first make some Poisson data with the following properties:
* length of the light curve: 16 seconds
* time steps in light curve: 1024\*16 time bins
* constant amplitude: 10 counts/bin

1. What is the count rate in counts per second?
2. make a time series with the property above, then add Poisson noise: for each value of your model, pick from a Poisson distribution with a rate parameter equal to the value of your model at that bin (Hint: the `numpy.random` module is your friend!)
3. Plot the results
4. Do the same for Gaussian noise with the same mean and variance and compare the two. Are the statistical properties (e.g. mean and variance) the same?

## Simulating a periodic signal

As a simple warm-up exercise, let's see what the power spectrum of a simple sine wave looks like. Use the following parameters:
* length of the light curve: 16 seconds
* time steps in light curve: 1024\*16 time bins
* period of the sine wave: 0.01
* constant background amplitude: 10 counts/bin

Here are the steps you need to go through:
1. Define a sine wave
2. Compute the value of the sine wave + background at each time step
3. For a realistic data set, we'll include Poisson counting statistics: for each value of your model, pick from a Poisson distribution with a rate parameter equal to the value of your model at that bin.
4. Plot the result

Doesn't look like much, does it? 

You now have time bins, and counts per bin for each of those time bins. Let's now make a power spectrum.
The power spectrum is the square of the Fourier transform (which is implemented in both `numpy` and `scipy`). However, in Stingray, we've already implemented the power spectrum as it is often used in (time series) astronomy. In order to use the Stingray definition, you'll first need to put your light curve into a `Lightcurve` object. This is a structure that's designed to help you with common tasks you might do with light curves.

You can call `Lightcurve` like this:

    In []: from stingray import Lightcurve
    In []: lc = Lightcurve(time, counts)
    
Typing `Lightcurve(` and pressing shift-tab should give you access to the docstring, which documents the common features of this object.

In [ ]:
from stingray import Lightcurve, Powerspectrum

Once you've done this, you can compute the power spectrum using the `Powerspectrum` class in Stingray:

    In []: from stingray import Powerspectrum
    In []: ps = Powerspectrum(lc)
    
By default, this object uses something called the fractional rms normalization. This is useful, because if you multiply each power by its frequencies, you can add up powers in any frequency range in order to directly compute the variability in the light curve at those time scales. 

However, for characterizing our sine wave, the Leahy normalization is more useful (because we know exactly what the statistical distribution for pure Poisson noise will be). So go ahead and when calling `Powerspectrum`, use the parameter `norm="leahy"` in order to tell the code to use Leahy normalization instead. You can look at the docstring for `Powerspectrum` (hint: in Jupyter notebooks, type `Powerspectrum()`, move your cursor between the brackets, and press tab-shift) to find out what other normalizations and parameters are available.


**Question**: Without looking at the result, you should be able to tell what the first frequency will be, 
what the last frequency will be, and how many elements your power spectrum has. Enter your answers below.

* $\nu_{\mathrm{min}} = $
*  $\nu_{\mathrm{max}} = $
* $n_{\mathrm{frequencies}} = $

Now check the result. You can access the frequency array (at the centre of each frequency bin) as `ps.freq` (assuming you named your `Powerspectrum` object `ps`, otherwise substitute whatever name you gave it!).

The number of frequencies should be off by one from your theoretical prediction. Why? Because in Stingray, we have decided to *exclude* the zeroth frequency (the first element in the array) from the power spectrum. The reason for this is that the zeroth frequency is literally the power at a frequency of zero. In practical terms, this is also called the DC (direct current) component, and in Leahy normalization is always equal to the square of the number of photons in your power spectrum. Therefore, it doesn't add any information on the timing, which is why we exclude it (it is usually excluded in analyses, anyway). 

The minimum frequency should match, while the Nyquist frequency is slightly off due to the definition we used for the frequency array in terms of the center of the frequency bins rather than the edges.

In order to plot the result, you can use the attributes `freq`, which encodes the array of frequencies (at the centre of each frequency bin), and `power`, which encodes the array of corresponding powers.

You can try plotting in linear space or in log-log space. Try both. Under what circumstances do you think the log-log plot could be useful?

You should see a huge spike and some noise. Do you? That's our periodic signal. Check that it is exactly at $1/p$, where $p$ is the period of your sine wave, by zooming into the plot.

Do you remember what the statistical distribution of the noise should be? If not, you can look up the distribution of a random variable that's defined as the sum of two normally distributed random variables.

Look at only the powers to the right of the periodic signal. Check whether they're distributed as you would expect given your answer above (if you didn't have a chance to figure it out, the solution is a $\chi^2$ distribution with 2 degrees of freedom). 

* Try plotting a (normalized) histogram (`matplotlib.pyplot.hist`) of your samples along with the PDF of the correct distribution
* Do a comparison between your sample and the correct distribution using a Kolmogorov-Smirnov test (also implemented in `scipy.stats`, it takes your array of noise powers and the cdf of a distribution implemented in scipy.stats as input).

Hint: The correct distribution is defined in `scipy.stats`. 

The KS-test p-value should hopefully be quite large. This indicates that we cannot reject the null hypothesis (our null hypothesis is that our powers are distributed following a $\chi^2$ distribution with two degrees of freedom). This is, strictly speaking, not proof that the null hypothesis is true, but in practice, we have many powers in our noise power sample (the test becomes more discriminative as the sample size increases), and we expect a $\chi^2$ distribution to be the underlying distribution of our power spectrum. It's probably a pretty good assumption that the powers actually follow that distribution.

Okay, cool. For your next exercise, try out different combinations of the period amplitude and background count rate. 

* What happens if you make the amplitude much larger than the background count rate? (Hint: This should produce an error; think about why this produces an error)
* What happens to the power spectrum if the background count rate and period amplitude is really small? How large must the background counts and the amplitude be for you to see the periodic signal by eye? 
* What happens to the power spectrum if the background is large, but the period amplitude is small?
* Does the height of the peak in the power spectrum only depend on the absolute value of the amplitude, or does it depend on the ratio of the two?

### Assessing the significance of a periodic signal

To formally assess the significance of a periodic signal, you can use the formalism laid out in [Groth, 1975](http://adsabs.harvard.edu/abs/1975ApJS...29..285G) (don't read this paper unless you're actually interested in the intricate details of how to find a periodic signal in noise). The gist of the paper is that it computes the probability of obtaining a power as high as the one in question under the null hypothesis that there is no periodic signal in the data, while also taking into account that there *could* be a periodic signal, but it's so weak that you miss it. Again, in principle, the resulting p-value only allows you to reject the null hypothesis that the power spectrum is only white noise. In practice, *if* your light curve is strictly constant, and *if* the only alternative possibility is a periodic signal, then you may be able to conclude that there is a periodic signal if the null hypothesis is rejected. Note: This is formally incorrect! However, it works well in practice (though the literature also contains claims of periods that are clearly not correct), and someone has yet to come up with a fully Bayesian solution to this problem (to my knowledge).

In practice, as we've seen in the lecture, the assumption of a constant light curve is very often broken, making the analysis much more complex.

Here, however, let's assume it's true (and in fact, because we've simulated the data, we know it's true!). Let's make another data set with the following properties and then assess its significance:

In [ ]:
bkg = 10.0
period = 0.01
amplitude = 0.3


The formalism in Groth (1975) is implemented in Stingray as a method of `Powerspectrum`. You can call it like this:

    In []: res = ps.classical_significances()
    
The result is a list with two elements. The first is a list of indices, where each index is the index of the frequency bin, and a list of corresponding p-values for the null hypothesis that there is no periodic signal in the data. Essentially, this function looks at each frequency, then computes the probability of seeing that power under the null hypothesis, and computes the p-value for the power at this frequency. 

The method takes a keyword `threshold`, which sets the significance level for p-values to be reported. For example, if you set a threshold of 0.01, only p-values smaller than 0.01 will be in the list returned. This allows you to do easy significance tests. By default, the threshold is set to 1, which means that all p-values will be reported.

* What would you consider a good/useful threshold for using here? Why? Use this threshold. How many p-values are reported? 
* Try out different thresholds. How many p-values are reported for a threshold of [0.1, 0.05, 0.01, 0.001, 1e-6]?
* Say you set a significance level of 0.01. That means that in principle, under the assumptions made above, you consider all frequencies where a p-value is $<0.01$ to contain a periodic signal. Given what you know about the data you simulated, do you think that is reasonable? Why (not)?
* What could you do to make your inference more robust to this problem? Hint: you may want to look at the docstring of the `classical_significances` method.

As a last step, we'd like to know the fractional rms amplitude of this periodic signal. The fractional rms amplitude tells you the fraction of the total variance that's contained in your periodic signal. 

* Recompute the power spectrum, but set `norm="frac"`. Plot this and compare it to the power spectrum in Leahy normalization. What has changed?
* The fractional rms is defined as the standard deviation divided by the mean of your light curve. In fractional rms normalization, this should equal the integral of the power spectrum. Check that this is indeed the case. Hint: In rms normalization, this is easily computed by multiplying your powers by the frequency resolution and adding them all up. Be aware, however, that you should only add *one half* of the last frequency bin.
* Compute the fractional rms of the periodic signal.


## Finding A Periodic Signal

Okay, having done this for data we've simulated ourselves, you now get to do this on a data set you *don't* know. 
Using everything we've learned in the previous exercise, the goal is to assess whether there is a periodic signal in the data.

The data should be in the same folder as the notebook, and it's in a simple text file, so you can simply load it using `numpy`:

In [ ]:
times = np.loadtxt("periodsearch1.txt")

The data is a list of times of arrival: individual photon arrival times at the detector. This is *not* a light curve. It is not binned! Therefore, we'll need to bin it ourselves. Unfortunately, the scientist who gave you the data has not told you what instrument the data came from, and they're not answering e-mails. This means you don't know what the time resolution of the data is! You're going to have to guess.


* Take a look at the data. You can compute, for example, the total duration of the observation. You can also try to see if you can infer the time resolution from the data (Hint: a histogram of some derived quantity might help!)
* If you can't figure out the time resolution from the data, pick one. You can try different time resolutions. Hint: Stingray allows you to make a light curve out of times of arrival data using the following syntax (where `dt` is your time resolution, and you can set the duration of your observation `tseg` and the start time of your observation `tstart` by hand, if you know it):

        In []: Lightcurve.make_lightcurve(times, dt, tseg=None, tstart=None)

* Now compute the power spectrum of the light curve you just created. Is there a period in the data?
* Compare your results with your neighbours. Do you all get the same results? If yes, try out different time resolutions: try a time resolution that's a multiple of your current time resolution. Try a time resolution that's 1.2 times your current time resolution. Do you still get the same results? 
* Think about why you might have gotten the results you've seen (This one is difficult; you might want to look at the raw times of arrival again, and additionally look up *aliasing*).